In [ ]:
import pandas as pd
import os
import sys

In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [ ]:
BW_PROJECT = 'metallican' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

# Regioinvent

In [ ]:
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append(r"C:\Users\mp_ma\OneDrive - polymtl\POST_DOC\Modules\Regioinvent\src")
import regioinvent

In [ ]:
regio = regioinvent.Regioinvent(bw_project_name='metallican', ecoinvent_database_name='ecoinvent-3.10-cutoff', ecoinvent_version='3.10')

In [ ]:
regio.spatialize_my_ecoinvent()

In [ ]:
regio.import_fully_regionalized_impact_method(lcia_method='all')

In [ ]:
regio.regionalize_ecoinvent_with_trade(trade_database_path=r'C:\Users\mp_ma\OneDrive - polymtl\POST_DOC\Modules\Regioinvent\trade_data.db',
                                       regioinvent_database_name='Regioinvent',
                                       cutoff=0.99)

# Custom LCIs

In [ ]:
lci_brgm_path = r'not_in_github/Lai_et_al'
lci_premise_path = r'data/LCI/from_premise'
lci_istrate_path = r'data/LCI/Istrate_2024'

## From BRGM

In [ ]:
lci_brgm_files = [
    "Ag_LCIs_BW_Final_v2.xlsx",
    "Al_LCIs_BW_Final_v1.xlsx",
    "Au_LCIs_BW_Final_v1.xlsx",
    #"Cd_LCIs_BW_Final_v1.xlsx",
    "Cu_LCIs_BW_Final_v2.xlsx",
    "Fe_LCIs_BW_Final_v1.xlsx",
    #"Mo_LCIs_BW_Final_v1.xlsx",
    "Ni_LCIs_BW_Final_v1.xlsx",
    "Zn_LCIs_BW_Final_v2.xlsx",
    #"Te_LCIs_BW_Final_v2.xlsx",
]

In [ ]:
for filename in lci_brgm_files:
    filepath = os.path.join(lci_brgm_path, filename)
    print(f"Importing: {filename}")

    importer = bw.ExcelImporter(filepath)
    importer.apply_strategies()

    importer.match_database("ecoinvent-3.10-cutoff", fields=("name", "reference product", "unit", "location"))
    importer.match_database("biosphere3", fields=("name", "unit", "categories"))

    unlinked = list(importer.unlinked)
    print(f"  → Unlinked exchanges: {len(unlinked)}")

    if not unlinked:
        importer.write_database()
        print(f"  → Database '{importer}' written.")
    else:
        print(f"  → Skipping '{filename}' due to unlinked exchanges.")

## From Premise

In [ ]:
lci_premise_files = [
    "lci-sulfidic-tailings.xlsx",
    "lci-steel.xlsx"
]

In [ ]:
for filename in lci_premise_files:
    filepath = os.path.join(lci_premise_path, filename)
    print(f"Importing: {filename}")

    importer = bw.ExcelImporter(filepath)
    importer.apply_strategies()

    importer.match_database("ecoinvent-3.10-cutoff", fields=("name", "reference product", "unit", "location"))
    importer.match_database("biosphere3", fields=("name", "unit", "categories"))

    unlinked = list(importer.unlinked)
    print(f"  → Unlinked exchanges: {len(unlinked)}")

    if not unlinked:
        importer.write_database()
        print(f"  → Database '{importer}' written.")
    else:
        print(f"  → Skipping '{filename}' due to unlinked exchanges.")

In [ ]:
# LCI from ??
tailings = 'data/LCI/from_premise/lci-sulfidic-tailings.xlsx'
tailings = bi.ExcelImporter(tailings)
# Apply the necessary strategies
tailings.apply_strategies()

In [ ]:
# Let's try to link them with EI
tailings.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
tailings.match_database("biosphere3", fields=('name', 'unit', 'categories'))
tailings.statistics()

In [ ]:
[u for u in tailings.unlinked if u["type"] == "technosphere"]

In [ ]:
[u for u in tailings.unlinked if u["type"] == "biosphere"]

In [ ]:
migration_tailings = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("cement production, blast furnace slag 70-100%",
             "cement, blast furnace slag 70-100%",
             "US"),
            {"name": "cement production, type S",
             "reference product": "cement, type S"}
        ),

        (
            ("treatment of wastewater, average, capacity 1E9l/year",
             "wastewater, average",
             "Europe without Switzerland"),
            {"name": "treatment of wastewater, average, wastewater treatment"}
        ),

        (
            ("market for sodium hydroxide, without water, in 50% solution state",
             "sodium hydroxide, without water, in 50% solution state",
             "GLO"),
            {"location": "RoW"}
        ),
    ],
}

In [ ]:
bi.Migration(name="tailings").write(data=migration_tailings, description="ei 3.8 to 3.10")

In [ ]:
tailings.data = bi.strategies.migrate_exchanges(
    db=tailings.data,
    migration="tailings"
)

In [ ]:
tailings.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
tailings.match_database("biosphere3", fields=('name', 'unit', 'categories'))
tailings.statistics()

In [ ]:
if len(list(tailings.unlinked)) == 0:
    tailings.write_database()

## From Istrate et al (2024)

In [ ]:
# # Import LIB raw materials LCIs
# lci_lib_rms = bw.ExcelImporter(r'data/LCI/lci_LIB_raw_materials.xlsx')
# lci_lib_rms.apply_strategies()
# lci_lib_rms.match_database("ecoinvent-3.9.1-cutoff", fields=('name', 'reference product', 'unit', 'location'))
# lci_lib_rms.match_database("biosphere3", fields=('name', 'unit', 'categories'))
# # Add uncertainty data for pedigree matrix
# for ds in lci_lib_rms.data:
#     for exc in ds["exchanges"]:
#         if "pedigree" in exc:
#             # Pedigree are stored as strings
#             pedigree_str = exc["pedigree"].strip("()")
#             pedigre_scores = tuple([int(x) for x in pedigree_str.split(", ")])
#             exc_amount = exc["amount"]
#
#             uncertainty_dict = core.lca_calculation_functions.create_pedigree_matrix(pedigre_scores, exc_amount)
#             exc.update(uncertainty_dict)
# lci_lib_rms.statistics()
# [u for u in lci_lib_rms.unlinked if u["type"] == "biosphere"]
# [u for u in lci_lib_rms.unlinked if u["type"] == "technosphere"]

# Export EI LCIs for selected activities

In [ ]:
from core.lca_calculation_functions import get_inventory_dataset, export_activity_exchanges

In [ ]:
EI_DB = 'ecoinvent-3.10-cutoff'

In [ ]:
INVENTORIES_EI = {
    "Copper concentrate": ("gold-silver mine operation and beneficiation", "copper concentrate, sulfide ore", 'CA-QC'),
    "Iron concentrate": ("iron ore mine operation and beneficiation", "iron ore concentrate", "CA-QC"),
    "Gold-silver ingot": ("gold-silver mine operation and beneficiation", "gold-silver, ingot", "CA-QC"),
    "Gold": ("gold-silver mine operation with refinery", "gold", "CA-QC"),
    "Silver": ("gold-silver mine operation with refinery", "silver", "CA-QC"),
    "Nickel concentrate": ("nickel mine operation and benefication to nickel concentrate, 16% Ni", "nickel concentrate, 16% Ni", "CA-QC"),
}

In [ ]:
INVENTORIES_EI_ds = get_inventory_dataset(INVENTORIES_EI, database_names=[EI_DB])

In [ ]:
INVENTORIES_EI_ds

In [ ]:
# 2️⃣ Export their technosphere & biosphere flows
export_activity_exchanges(INVENTORIES_EI_ds)